In [9]:
from dataclasses import dataclass
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.runtime import get_runtime
from langchain_ollama import ChatOllama
import os
# Define system prompt
system_prompt = """You are an expert weather forecaster, who speaks in puns.

You have access to two tools:

- get_weather_for_location: use this to get the weather for a specific location
- get_user_location: use this to get the user's location

If a user asks you for the weather, make sure you know the location. If you can tell from the question that they mean wherever they are, use the get_user_location tool to find their location."""

# Define context schema
@dataclass
class Context:
    """Custom runtime context schema."""
    user_id: str

# Define tools
def get_weather_for_location(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

def get_user_location() -> str:
    """Retrieve user information based on user ID."""
    runtime = get_runtime(Context)
    user_id = runtime.context.user_id
    return "Florida" if user_id == "1" else "SF"

# Configure model
#model=ChatOpenAI(base_url="https://api.deepseek.com", model="deepseek-chat",api_key=os.environ['DEEPSEEK_API_KEY'])
model = ChatOllama(model="gpt-oss:120b-cloud")
# Define response format
@dataclass
class ResponseFormat:
    """Response schema for the agent."""
    # A punny response (always required)
    punny_response: str
    # Any interesting information about the weather if available
    weather_conditions: str | None = None

# Set up memory
checkpointer = InMemorySaver()

# Create agent
agent = create_agent(
    model=model,
    system_prompt=system_prompt,
    tools=[get_user_location, get_weather_for_location],
    context_schema=Context,
    response_format=ResponseFormat,
    checkpointer=checkpointer
)

# Run agent
# `thread_id` is a unique identifier for a given conversation.
config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather outside?"}]},
    config=config,
    context=Context(user_id="1")
)

print(response['structured_response'])
# ResponseFormat(
#     punny_response="Florida is still having a 'sun-derful' day! The sunshine is playing 'ray-dio' hits all day long! I'd say it's the perfect weather for some 'solar-bration'! If you were hoping for rain, I'm afraid that idea is all 'washed up' - the forecast remains 'clear-ly' brilliant!",
#     weather_conditions="It's always sunny in Florida!"
# )


# Note that we can continue the conversation using the same `thread_id`.
response = agent.invoke(
    {"messages": [{"role": "user", "content": "thank you!"}]},
    config=config,
    context=Context(user_id="1")
)

print(response['structured_response'])
# ResponseFormat(
#     punny_response="You're 'thund-erfully' welcome! It's always a 'breeze' to help you stay 'current' with the weather. I'm just 'cloud'-ing around waiting to 'shower' you with more forecasts whenever you need them. Have a 'sun-sational' day in the Florida sunshine!",
#     weather_conditions=None
# )

ResponseFormat(punny_response="Looks like the Sunshine State is living up to its name—it's a bright, sunny day outside! 🌞", weather_conditions='Sunny')
ResponseFormat(punny_response="Looks like the Sunshine State is living up to its name—it's a bright, sunny day outside! 🌞", weather_conditions='Sunny')


dynamic model

In [3]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_model_call, ModelRequest
from langchain.agents.middleware.types import ModelResponse #langchain bug,后续他们会把modelresponse放到langchain.agents.middleware里
from langchain_ollama import ChatOllama
import os
basic_model=ChatOpenAI(base_url="https://api.deepseek.com", model="deepseek-chat",api_key=os.environ['DEEPSEEK_API_KEY'])
advanced_model=ChatOllama(model="gpt-oss:120b-cloud")

def search_user_location() -> str:
    """get user location"""
    return "Suzhou"

def get_weather_for_location(city: str) -> str:
    """get weather for a given city."""
    return f"It's always sunny in {city}!"

@wrap_model_call
def dynamic_model_middleware(request: ModelRequest, handler) -> ModelResponse:
    message_count=len(request.state["messages"])
    if message_count > 10:
        model=advanced_model
    else:
        model=basic_model
    request.model=model
    return handler(request)

agent = create_agent(model=basic_model,
                     tools=[get_weather_for_location,search_user_location],
                     middleware=[dynamic_model_middleware]
                     )
response=agent.invoke({"messages": [{"role": "user", "content": "what is the weather outside,hahahahahahahahahaha?"}]})
response

{'messages': [HumanMessage(content='what is the weather outside,hahahahahahahahahaha?', additional_kwargs={}, response_metadata={}, id='850d9f96-f87d-425e-8859-603c8d9163b1'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 188, 'total_tokens': 197, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 128}, 'prompt_cache_hit_tokens': 128, 'prompt_cache_miss_tokens': 60}, 'model_provider': 'openai', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': '43a1eeca-8cea-4eb8-980a-83fc7df8ae15', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--269e9ba9-4cf1-4628-b1f7-6eb38312d1ae-0', tool_calls=[{'name': 'search_user_location', 'args': {}, 'id': 'call_00_08HMO3Vk1oO60Rvf7FsNp4Mo', 'type': 'tool_call'}], usage_metadata={'input_tokens': 188, 'output_tokens': 9, 'total_tokens': 197, 

tool error handling

In [5]:
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_tool_call
from langchain_core.messages import AIMessage, HumanMessage,ToolMessage
from langchain_ollama import ChatOllama
@wrap_tool_call
def handle_tool_errors(request, handler):
    """ Handle tool execution errors with custom messages"""
    try:
        response=handler(request)
        return response
    except Exception as e:
        return ToolMessage(
            content=f"Tool error: Please check your input and try again. ({str(e)})",
            tool_call_id=request.tool_call["id"]
        )
advanced_model=ChatOllama(model="gpt-oss:120b-cloud")
agent = create_agent(model=advanced_model,tools=[get_weather_for_location,search_user_location],middleware=[handle_tool_errors])
response=agent.invoke({"messages": [{"role": "user", "content": "what is the weather outside?"}]})

ImportError: cannot import name 'wrap_tool_call' from 'langchain.agents.middleware' (d:\SourceCode-2\langchain-alpha1\.venv\Lib\site-packages\langchain\agents\middleware\__init__.py)

dynamic system prompt

In [ ]:
from typing import TypedDict
from langchain.agents import create_agent
from langchain.agents.middleware import dynamic_prompt, ModelRequest

class Context(TypedDict):
    user_id: str

@dynamic_prompt
def user_role_prompt(request: ModelRequest) -> str:
    """Generate system prompt based on user role."""
    user_role=request.runtime.context.get("user_role","user")
    base_prompt="Your are a helpful assistant."
    if user_role == "expert":
        return f"{base_prompt} Provide detailed technical responses."
    elif user_role == "beginner":
        return f"{base_prompt} Explain concepts simply and avoid jargon."
    return base_prompt

agent= create_agent(model=advanced_model,tools=[get_weather_for_location,search_user_location],middleware=[user_role_prompt],context_schema=Context)

result=agent.invoke({"messages": [{"role": "user", "content": "what is the weather outside?"}]},context=Context(user_id="1",user_role="expert"))
result

{'messages': [HumanMessage(content='what is the weather outside?', additional_kwargs={}, response_metadata={}, id='552b5b67-eca8-4663-a3b7-7b0db962c9a9'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'gpt-oss:120b-cloud', 'created_at': '2025-10-13T08:05:55.670559981Z', 'done': True, 'done_reason': 'stop', 'total_duration': 895329161, 'load_duration': None, 'prompt_eval_count': 145, 'prompt_eval_duration': None, 'eval_count': 58, 'eval_duration': None, 'model_name': 'gpt-oss:120b-cloud', 'model_provider': 'ollama'}, id='lc_run--b3df58a1-756b-4438-9bed-c76036778ba8-0', tool_calls=[{'name': 'search_user_location', 'args': {}, 'id': '0fdb2047-6154-411d-ae1e-e457c3e73020', 'type': 'tool_call'}], usage_metadata={'input_tokens': 145, 'output_tokens': 58, 'total_tokens': 203}),
  ToolMessage(content='Suzhou', name='search_user_location', id='8cfa203c-3eba-4d38-930c-5fe9d1c65ddb', tool_call_id='0fdb2047-6154-411d-ae1e-e457c3e73020'),
  AIMessage(content='', addition

toolstrategy

In [2]:
from pydantic import BaseModel
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy

class ContactInfo(BaseModel):
    email: str
    phone: str
    name:str

agent= create_agent(model=advanced_model,tools=[],response_format=ToolStrategy(ContactInfo))
result=agent.invoke({"messages": [{"role": "user", "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567"}]})
result["structured_response"]

ContactInfo(email='john@example.com', phone='(555) 123-4567', name='John Doe')

memory

In [3]:
from typing import Annotated,TypedDict
from langchain.agents import create_agent,AgentState
from langchain.agents.middleware import AgentMiddleware
from langchain_ollama import ChatOllama
advanced_model=ChatOllama(model="gpt-oss:120b-cloud")
class CustomAgentState(AgentState):
    user_preferences:dict

class PreferenceMiddleware(AgentMiddleware[CustomAgentState]):
    state_schema=CustomAgentState

agent=create_agent(advanced_model,tools=[],middleware=[PreferenceMiddleware()])
result = agent.invoke({
    "messages": [{"role": "user", "content": "I prefer technical explanations"}],
    "user_preferences": {"style": "technical", "verbosity": "detailed"},
})
result

{'messages': [HumanMessage(content='I prefer technical explanations', additional_kwargs={}, response_metadata={}, id='2c8a12ee-726e-4bed-9d2c-da3be1e1f1f2'),
  AIMessage(content='Got it! I’ll keep my answers technical and dive into the details. Let me know what topic or question you’d like to explore.', additional_kwargs={}, response_metadata={'model': 'gpt-oss:120b-cloud', 'created_at': '2025-10-13T08:53:36.081781738Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2735229264, 'load_duration': None, 'prompt_eval_count': 78, 'prompt_eval_duration': None, 'eval_count': 82, 'eval_duration': None, 'model_name': 'gpt-oss:120b-cloud', 'model_provider': 'ollama'}, id='lc_run--c0f7f0c9-e8e6-4f1d-9d29-648a6aabc747-0', usage_metadata={'input_tokens': 78, 'output_tokens': 82, 'total_tokens': 160})],
 'user_preferences': {'style': 'technical', 'verbosity': 'detailed'}}

before model

In [6]:
from typing import Any
from langchain.messages import RemoveMessage
from langgraph.graph.message import REMOVE_ALL_MESSAGES
from langchain.agents import create_agent,AgentState
from langchain.agents.middleware import before_model
from langgraph.runtime import Runtime
@before_model
def trim_messages(state:AgentState,runtime:Runtime)->dict[str,Any]|None:
    messages=state["messages"]
    print(len(messages))
    if(len(messages)<=3):
        return None
    
    first_msg=messages[0]
    recent_messages=messages[-3:] if len(messages)%2==0 else messages[-4:]
    new_messages=[first_msg]+recent_messages
    return {"messages":[RemoveMessage(id=REMOVE_ALL_MESSAGES),*new_messages
                        
                        ]}

agent=create_agent(advanced_model,tools=[],middleware=[trim_messages])
response=agent.invoke({"messages":[{"role":"user","content":"hello"},{"role":"assistant","content":"hi"},{"role":"user","content":"how are you?"},{"role":"assistant","content":"fine"}]})
response

4


{'messages': [HumanMessage(content='hello', additional_kwargs={}, response_metadata={}, id='f7a41365-c41f-4c28-9971-b145b8d668ad'),
  AIMessage(content='hi', additional_kwargs={}, response_metadata={}, id='6268eca1-7abd-4c0c-bc59-5be38dc23ab0'),
  HumanMessage(content='how are you?', additional_kwargs={}, response_metadata={}, id='2c6561c5-0666-4c4d-b0bb-2bdf3ea46fca'),
  AIMessage(content='fine', additional_kwargs={}, response_metadata={}, id='41c52111-b53d-4cdb-b05f-f9f1a8df9fd3'),
  AIMessage(content="I'm doing well, thank you! How can I assist you today?", additional_kwargs={}, response_metadata={'model': 'gpt-oss:120b-cloud', 'created_at': '2025-10-13T09:12:56.010450345Z', 'done': True, 'done_reason': 'stop', 'total_duration': 586246797, 'load_duration': None, 'prompt_eval_count': 95, 'prompt_eval_duration': None, 'eval_count': 36, 'eval_duration': None, 'model_name': 'gpt-oss:120b-cloud', 'model_provider': 'ollama'}, id='lc_run--afeaf111-e50c-45fb-ad9a-c84b2d4bb3b5-0', usage_meta

after model

In [ ]:
from typing import Any
from langchain.messages import AIMessage, RemoveMessage
from langgraph.graph.message import REMOVE_ALL_MESSAGES
from langchain.agents import create_agent, AgentState
from langchain.agents.middleware import after_model
from langgraph.runtime import Runtime

@after_model
def validate_response(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    """Check model response for policy violations."""
    messages = state["messages"]
    last_message = messages[-1]

    if "confidential" in last_message.content.lower():
        return {
            "messages": [
                RemoveMessage(id=REMOVE_ALL_MESSAGES),
                *messages[:-1],
                AIMessage(content="I cannot share confidential information.")
            ]
        }

    return None  # No changes needed

agent = create_agent(
    model=advanced_model,
    tools=[],
    middleware=[validate_response]
)

stream

In [7]:
for chunk in agent.stream({
    "messages": [{"role": "user", "content": "Search for AI news and summarize the findings"}]
}, stream_mode="values"):
    # Each chunk contains the full state at that point
    latest_message = chunk["messages"][-1]
    if latest_message.content:
        print(f"Agent: {latest_message.content}")
    elif latest_message.tool_calls:
        print(f"Calling tools: {[tc['name'] for tc in latest_message.tool_calls]}")

Agent: Search for AI news and summarize the findings
1
Agent: **What I can do**

I don’t have live‑web browsing capabilities, so I can’t pull down the very latest headlines from today’s internet. However, I keep a fairly up‑to‑date internal knowledge base that includes major AI developments up through June 2024. Below is a concise summary of the most notable AI news and trends from roughly the past year (mid‑2023 → mid‑2024). If you’d like more detail on any of these items—or if you have a specific time window or topic in mind—just let me know and I can dig deeper.

---

## 1. **Foundation Model Proliferation & Competition**

| Development | Why It Matters | Key Players |
|------------|----------------|-------------|
| **OpenAI releases GPT‑4 Turbo (Oct 2023) and GPT‑4o (July 2024).** The “Turbo” variant is cheaper and faster, while GPT‑4o adds higher‑fidelity multimodal capabilities (audio, video, and real‑time translation). | Sets new performance‑price benchmarks; drives enterprise a

tool calling

In [4]:
from langchain.tools import tool
from langchain_ollama import ChatOllama
model=ChatOllama(model="gpt-oss:120b-cloud")

@tool
def get_weather(location:str)->str:
    """Get the weather at a location"""
    return f"It's always sunny in {location}!"

model_with_tools=model.bind_tools([get_weather])

response=model_with_tools.invoke("What's the weather like in Boston")
for tool_call in response.tool_calls:
    print(f"Tool: {tool_call['name']}")
    print(f"Args: {tool_call['args']}")

Tool: get_weather
Args: {'location': 'Boston'}


tool execution loop

In [7]:
# Bind (potentially multiple) tools to the model
model_with_tools = model.bind_tools([get_weather])

# Step 1: Model generates tool calls
messages = [{"role": "user", "content": "What's the weather in Boston?"}]
ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)
print(ai_msg)
# Step 2: Execute tools and collect results
for tool_call in ai_msg.tool_calls:
    # Execute the tool with the generated arguments
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)

print(messages)
# Step 3: Pass results back to model for final response
final_response = model_with_tools.invoke(messages)
print(final_response.text)
# "The current weather in Boston is 72°F and sunny."

content='' additional_kwargs={} response_metadata={'model': 'gpt-oss:120b-cloud', 'created_at': '2025-10-14T03:11:20.576535952Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3392451783, 'load_duration': None, 'prompt_eval_count': 129, 'prompt_eval_duration': None, 'eval_count': 56, 'eval_duration': None, 'model_name': 'gpt-oss:120b-cloud', 'model_provider': 'ollama'} id='lc_run--01c87f23-9fa1-4881-8ea8-e6154aaa806f-0' tool_calls=[{'name': 'get_weather', 'args': {'location': 'Boston'}, 'id': '7d75c192-c36a-4572-add1-2582307a8bfd', 'type': 'tool_call'}] usage_metadata={'input_tokens': 129, 'output_tokens': 56, 'total_tokens': 185}
[{'role': 'user', 'content': "What's the weather in Boston?"}, AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'gpt-oss:120b-cloud', 'created_at': '2025-10-14T03:11:20.576535952Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3392451783, 'load_duration': None, 'prompt_eval_count': 129, 'prompt_eval_duration': None, '

structured outputs

pydantic

In [3]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
import os
#model=ChatOpenAI(base_url="https://api.deepseek.com", model="deepseek-chat",api_key=os.environ['DEEPSEEK_API_KEY'])
model=ChatOllama(model="deepseek-r1:7b")
class Movie(BaseModel):
    """A movie with details."""
    title: str = Field(..., description="The title of the movie")
    year: int = Field(..., description="The year the movie was released")
    director: str = Field(..., description="The director of the movie")
    rating: float = Field(..., description="The movie's rating out of 10")

model_with_structure = model.with_structured_output(Movie)
response = model_with_structure.invoke("Provide details about the movie Inception")
print(response)  # Movie(title="Inception", year=2010, director="Christopher Nolan", rating=8.8)

title='Inception' year=2010 director='Christopher Nolan' rating=8.647593582879953


In [6]:
response["title"]

'Inception'

typedDict

In [8]:
from typing_extensions import TypedDict,Annotated
from langchain_ollama import ChatOllama
model=ChatOllama(model="deepseek-r1:7b")

class MovieDict(TypedDict):
    """A movie with details."""
    title: Annotated[str, ..., "The title of the movie"]
    year: Annotated[int, ..., "The year the movie was released"]
    director: Annotated[str, ..., "The director of the movie"]
    rating: Annotated[float, ..., "The movie's rating out of 10"]
model_with_structure = model.with_structured_output(MovieDict)
response = model_with_structure.invoke("Provide details about the movie Inception")
print(response)  # {'title': 'Inception', 'year': 2010,

{'title': 'Inception', 'year': 502, 'director': 'Christopher Nolan', 'rating': 4.6}


In [9]:
response["title"]

'Inception'

multimodal

In [12]:
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
model=ChatOpenAI(base_url="https://api.deepseek.com", model="deepseek-chat",api_key=os.environ['DEEPSEEK_API_KEY'])
#model=ChatOllama(model="gpt-oss:120b-cloud")
response=model.invoke("Create a picture of a cat")
print(response)

content="I can't directly create images, but I can help you imagine or describe a picture of a cat! Here's a detailed description you can use with an AI image generator:\n\n**A realistic, detailed portrait of a cat:**\n- **Breed:** A fluffy Maine Coon cat\n- **Coat:** Soft, long fur with beautiful tabby markings in shades of gray, brown, and cream\n- **Eyes:** Large, expressive golden-amber eyes with dilated pupils\n- **Pose:** Sitting gracefully, head slightly tilted, with one paw gently raised\n- **Setting:** Curled up on a cozy windowsill with soft morning light streaming through\n- **Background:** Blurred living room with bookshelves and plants\n- **Details:** Whiskers catching the light, tiny nose is pink, ears perked up alertly\n\n**If you'd like to generate this image**, you could use:\n- DALL-E 3 (through ChatGPT Plus)\n- Midjourney\n- Stable Diffusion\n- Other AI image generators\n\nJust copy the description above into your preferred image generator! Would you like me to modif

reasoning

In [13]:
for chunk in model.stream("Why do parrots have colorful feathers?"):
    reasoning_steps = [r for r in chunk.content_blocks if r["type"] == "reasoning"]
    print(reasoning_steps if reasoning_steps else chunk.text)

AttributeError: 'AIMessageChunk' object has no attribute 'content_blocks'

server-side tool use

In [16]:
from langchain_openai import ChatOpenAI
import os
from langchain_ollama import ChatOllama
model=ChatOllama(model="gpt-oss:120b-cloud")
#model=ChatOpenAI(base_url="https://api.deepseek.com", model="deepseek-chat",api_key=os.environ['DEEPSEEK_API_KEY'])
tool = {"type": "web_search"}
model_with_tools = model.bind_tools([tool])

response = model_with_tools.invoke("What was a positive news story from today?")
response.content_blocks

ValueError: Unsupported function

{'type': 'web_search'}

Functions must be passed in as Dict, pydantic.BaseModel, or Callable. If they're a dict they must either be in OpenAI function format or valid JSON schema with top-level 'title' and 'description' keys.

rate-limit

In [17]:
from langchain_core.rate_limiters import InMemoryRateLimiter
from langchain.chat_models import init_chat_model

rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.1,  # 1 request every 10s
    check_every_n_seconds=0.1,  # Check every 100ms whether allowed to make a request
    max_bucket_size=10,  # Controls the maximum burst size.
)

model = init_chat_model(
    model="gpt-5",
    model_provider="openai",
    rate_limiter=rate_limiter  
)

proxy configuration

In [ ]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(
    model="gpt-4o",
    openai_proxy="http://proxy.example.com:8080"
)

log probabilities

In [ ]:
model = init_chat_model(
    model="gpt-4o",
    model_provider="openai"
).bind(logprobs=True)

response = model.invoke("Why do parrots talk?")
print(response.response_metadata["logprobs"])

invocation config

In [ ]:
response = model.invoke(
    "Tell me a joke",
        config={
        "run_name": "joke_generation",      # Custom name for this run
        "tags": ["humor", "demo"],          # Tags for categorization
        "metadata": {"user_id": "123"},     # Custom metadata
        "callbacks": [my_callback_handler], # Callback handlers
    }
)

configuration models

In [ ]:
from langchain.chat_models import init_chat_model

configurable_model = init_chat_model(temperature=0)

configurable_model.invoke(
    "what's your name",
        config={"configurable": {"model": "gpt-5-nano"}},  # Run with GPT-5-Nano
)
configurable_model.invoke(
    "what's your name",
        config={"configurable": {"model": "claude-3-5-sonnet-latest"}},  # Run with Claude
)

In [2]:
from langchain_anthropic import ChatAnthropic
import os
# 推荐使用 Claude Sonnet 4（性价比最好）
llm = ChatAnthropic(
    model="claude-sonnet-4",
    anthropic_api_key=os.environ['ANTHROPIC_API_KEY'],
    temperature=0.7
)

# 使用
response = llm.invoke("你好，请介绍一下自己")
print(response.content)

BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Anthropic API. Please go to Plans & Billing to upgrade or purchase credits.'}, 'request_id': 'req_011CU6nLWcTz8uyny38hPMUr'}

In [4]:
import os
from langchain.chat_models import init_chat_model
llm=init_chat_model(
    model="anthropic:claude-3-5-sonnet-latest",
    anthropic_api_key=os.environ['ANTHROPIC_API_KEY'],
)
response = llm.invoke("你好，请介绍一下自己")
response

BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Anthropic API. Please go to Plans & Billing to upgrade or purchase credits.'}, 'request_id': 'req_011CU6o9h5pJ9LxkcLWJUQec'}